# Entrenamiento del modelo para interpretación de saltos de línea en el movimiento del ojo

## Importaciones

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import SGD
from keras.layers import Dropout
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input, decode_predictions   
from keras.preprocessing import image                                            
import numpy as np
from tensorflow.keras.backend import set_image_data_format
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

In [ ]:
import os
import cv2
from glob import glob
import pandas as pd

## Montaje de Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Carga de datos

In [ ]:
from numpy import load

# Cargamos el numpy array desde un fichero npy.
x_set_r = load('/path/Saturdays AI - Equipo ojo/dataset/x_set_right.npy')
y_set_r = load('/path/Saturdays AI - Equipo ojo/dataset/y_set_right.npy')
x_set_l = load('/path/Saturdays AI - Equipo ojo/dataset/x_set_left.npy')
y_set_l = load('/path/Saturdays AI - Equipo ojo/dataset/y_set_left.npy')

x_set = np.concatenate((x_set_r,x_set_l))
y_set = np.concatenate((y_set_r,y_set_l))

# Comprobamos las dimensiones.
print(len(x_set_r))
print(len(y_set_r))
print(len(y_set_l))
print(len(y_set_l))
print(len(x_set))
print(len(y_set))

In [ ]:
print(f"Dimensiones del conjunto total: x_set={x_set.shape},dimensiones y_set={len(y_set)}")
X_train_set, X_test_set, y_train_set, y_test_set = train_test_split(x_set, y_set, test_size=0.20, random_state=0)
print(f"Dimensiones del conjunto de entrenamiento: X_train_set={X_train_set.shape},dimensiones y_train_set={len(y_train_set)}")
print(f"Dimensiones del conjunto de test: X_test_set={X_test_set.shape},dimensiones y_train_set={len(X_test_set)}")

In [ ]:
# Número de saltos de línea en el conjunto de entrenamiento y el de validación.
print(y_train_set.sum(),len(y_train_set))
print(y_test_set.sum(),len(y_test_set))

## Creación del modelo

In [ ]:
# Creamos el modelo.

set_image_data_format('channels_first') # Especificamos que los canales son el primer componente de la imagen.

includeTop = False  # Permite que la priemra capa no sea totalmente conexa (que no todas las neuronas estén totalmente conectadas a las de la siguiente capa), 
                    # y permite cambiar el tamaño del input de la primera capa.

weights_base = None # Establecemos los pesos a None porque está entrenada con un banco de imágenes que no nos sirve.

modelo_base = InceptionResNetV2(weights=weights_base, include_top=includeTop, input_shape=(10,80,80))

In [ ]:
# Creamos un modelo secuencial y se añade InceptionResNetV2 y capas extra para tener dos clases finales.
modelo_final = Sequential()
modelo_final.add(modelo_base) # Generamos un nuevo modelo base ya que, si modificamos la entrada, no podemos modificar la salida del modelo.
                              # De esta manera, modificamos la entrada del modelo original 
                              # y luego hacemos una especie de embudo para pasar de 1000 outputs a 2.

modelo_final.add(Flatten()) # Intersección entre una capa convolucional y una densa. Aplana los valores.
modelo_final.add(Dense(units=512, activation='relu'))
#modelo_final.add(Dropout(0.05))
modelo_final.add(Dense(units=256, activation='relu'))
#modelo_final.add(Dropout(0.5)) # Dropout comentado ya que tapamos las neuronas del final, haciéndo el modelo inestable al tapar las neuronas relevantes.
modelo_final.add(Dense(units=2, activation='sigmoid'))

modelo_final.summary()

In [ ]:
# Utilizamos el optimizador de adam ya que se comporta bien con datasets grandes y el ratio de aprendizaje es variable.
modelo_final.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Comprobamos el tamaño de la entrada del modelo.
modelo_final.input_shape

In [ ]:
# Transormamos los conjuntos con la variable dependiente como categóricos ya que el modelo precisa este formato.
print(y_train_set)
y_train_set = to_categorical(y_train_set,2)
print(y_train_set)
y_test_set = to_categorical(y_test_set, 2)

#[1. 0.]-> no hay lectura
#[0. 1.]-> hay lectura

## Entrenamiento del modelo

In [ ]:
# Entrenamos el modelo.
epochs=20
history = modelo_final.fit(X_train_set, y_train_set, epochs=epochs, verbose=1, validation_data=(X_test_set, y_test_set))

## Análisis de resultados del entrenamiento

In [ ]:
# Obtenemos los resultados y la matriz de confusión para el modelo entrenado.
from sklearn import metrics

y_pred = modelo_final.predict(X_test_set)

test_results = modelo_final.evaluate(X_test_set, y_test_set, verbose=0)
print(test_results)

matrix = metrics.confusion_matrix(y_test_set.argmax(axis=1), y_pred.argmax(axis=1))

print(matrix)

In [ ]:
# Obtenemos un mapa de calor de los resultados reportados por el modelo.
import seaborn as sns

df_cm = pd.DataFrame(matrix, index = [i for i in [1,0]],
                  columns = [i for i in [1,0]])
plt.figure(figsize = (10,7))
sns.heatmap(df_cm, annot=True)

In [ ]:
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

y_pred_prob = modelo_final.predict(X_test_set)

y_pred = [int(i[1] > .5) for i in y_pred_prob]

y_test = [int(i[0]==0) for i in y_test_set]

data = {'y_test': y_test, 'y_pred': y_pred}

df = pd.DataFrame(data, columns=['y_test','y_pred'])

confusion_matrix = pd.crosstab(df['y_test'], df['y_pred'], rownames=['Actual'], colnames=['Predicted'])
print (confusion_matrix)


In [ ]:
train_results = modelo_final.evaluate(X_train_set, y_train_set, verbose=0)
print('Error (loss) sobre datos de entrenamiento: {0:.4f}'.format(train_results[0]))
print('Tasa de éxito sobre datos de entrenamiento: {0:.4f}'.format(train_results[1]))

# Resultados sobre validación.
test_results = modelo_final.evaluate(X_test_set, y_test_set, verbose=0)
print('Error (loss) sobre datos de TEST: {0:.4f}'.format(test_results[0]))
print('Tasa de éxito sobre datos de TEST: {0:.4f}'.format(test_results[1]))

# Evolucion del rendimiento del modelo.
plt.figure(1)
x = range(1,epochs+1)
plt.plot(x, history.history['accuracy'], label='train')
plt.plot(x, history.history['val_accuracy'], label='val')
plt.xticks(x)
plt.ylim(0.5, 1)
plt.legend(loc="lower right")
plt.show()

## Guardardado del modelo

In [ ]:
from keras.models import save_model
path = "/path/Saturdays AI - Equipo ojo/modeloEntrenado"
save_model(modelo_final,path)